In [14]:
import pandas as pd
import scanpy as sc
import numpy as np
np.random.seed(1)
import scipy.sparse as sp
from anndata import concat

from helical import Geneformer,GeneformerConfig

from functions import add_noise, add_perturbations

# 1. Preprocess scRNA data (only run once)

### Load counts matrix and preprocess

In [18]:
file_path = "../input/GSE144236_cSCC_counts.txt" #This .txt file contains the scRNA counts for each gene and cell
scc_counts_df = pd.read_csv(file_path, sep="\t", header=0)  #Open .txt file as pandas df, separate by \t and use first line as header. 
scc_counts_df_T = scc_counts_df.T #transpose so cells in rows and genes in columns 
scc_counts_df_T_dropped = scc_counts_df_T.drop(scc_counts_df_T.columns[[0, 1]] , axis=1) #remove two first columns corresponding to patient and tissue type (not genes)
gene_names = scc_counts_df_T_dropped.columns.tolist() #save gene names as a list for later
scc_counts_df_T_dropped


,RP11-34P13.7,AL627309.1,AP006222.2,RP4-669L17.10,RP11-206L10.3,RP11-206L10.2,RP11-206L10.9,FAM87B,LINC00115,FAM41C,...,BX072566.1,KIR2DL2,KIR3DL2.1,AL590523.1,CT476828.1,AC145205.1,BAGE5,CU459201.1,AC002321.2,AC002321.1
P1_Tumor_AAACCTGAGTCAAGCG,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
P1_Tumor_AAACCTGCAAATTGCC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
P1_Tumor_AAACCTGGTAGGAGTC,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
P1_Tumor_AAACGGGAGATGTAAC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
P1_Tumor_AAACGGGAGCTGCAAG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P10_Normal_TTTGTCAAGAGTCTGG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
P10_Normal_TTTGTCAAGTCGTTTG,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
P10_Normal_TTTGTCACAGTGGGAT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
P10_Normal_TTTGTCATCCAGGGCT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Convert dataframe into AnnData object with counts in a sparse matrix (csr)

In [3]:
scc_annData = sc.AnnData(scc_counts_df_T_dropped) #convert to annData object
scc_annData.X = sp.csr_matrix(scc_annData.X, dtype='float64') #convert counts from array to sparse matrix (csr) -> more efficient and required format
scc_annData

AnnData object with n_obs × n_vars = 48164 × 32738

### Load metadata

Open metadata which including patient, tumor type, cell type

In [8]:
scc_annotations_df = pd.read_csv(f'../input/GSE144236_patient_metadata_new.txt', sep='\t')
scc_annotations_df

,nCount_RNA,nFeature_RNA,patient,tum.norm,level1_celltype,level2_celltype,level3_celltype
P1_Tumor_AAACCTGAGTCAAGCG,40856,5206,P1,Tumor,Epithelial,Tumor_KC_Diff,Tumor_KC_Diff
P1_Tumor_AAACCTGCAAATTGCC,15511,3468,P1,Tumor,Epithelial,Keratinocyte,Keratinocyte
P1_Tumor_AAACCTGGTAGGAGTC,24652,4154,P1,Tumor,Epithelial,Tumor_KC_Basal,Tumor_KC_Basal
P1_Tumor_AAACGGGAGATGTAAC,27554,4334,P1,Tumor,Epithelial,Tumor_KC_Basal,Tumor_KC_Basal
P1_Tumor_AAACGGGAGCTGCAAG,24980,4822,P1,Tumor,Epithelial,Tumor_KC_Cyc,Tumor_KC_Cyc
...,...,...,...,...,...,...,...
P10_Normal_TTTGTCAAGAGTCTGG,5585,1659,P10,Normal,Epithelial,Normal_KC_Diff,Normal_KC_Diff
P10_Normal_TTTGTCAAGTCGTTTG,14207,3541,P10,Normal,LC,LC,LC
P10_Normal_TTTGTCACAGTGGGAT,10068,2540,P10,Normal,LC,LC,LC
P10_Normal_TTTGTCATCCAGGGCT,5223,1849,P10,Normal,LC,LC,LC


Add metadata to annData object with counts

In [9]:
scc_annData.obs_names = scc_annotations_df.index #define .obs names in annData object
scc_annData.obs = scc_annData.obs.merge(scc_annotations_df, left_index=True, right_index=True, how='left') #Add metadata
scc_annData.var['gene_name'] = gene_names #define gene names in .var
scc_annData

AnnData object with n_obs × n_vars = 48164 × 32738
    obs: 'nCount_RNA', 'nFeature_RNA', 'patient', 'tum.norm', 'level1_celltype', 'level2_celltype', 'level3_celltype'
    var: 'gene_name'

In [ ]:
scc_annData.write_h5ad("../input/preprocessed_data/scc_annData.h5ad") #save annData object

### Filter for Tumor and Healthy Keratinocyes

Include all Keratinocytes including tumor and non-tumor specific ones from both tumor samples and non-tumor samples

In [57]:
KC_scc = scc_annData[scc_annData.obs['level3_celltype'].isin(['Tumor_KC_Diff', 'Tumor_KC_Basal', 'Tumor_KC_Cyc', 'Normal_KC_Diff', 'Normal_KC_Basal', 'Normal_KC_Cyc', 'TSK' ])] 
KC_scc

View of AnnData object with n_obs × n_vars = 18359 × 32738
    obs: 'nCount_RNA', 'nFeature_RNA', 'patient', 'tum.norm', 'level1_celltype', 'level2_celltype', 'level3_celltype'
    var: 'gene_name'

In [58]:
KC_scc.write_h5ad("../input/preprocessed_data/KC_scc_annData.h5ad")


# 2. In-silico perturbations

Load preprocessed dataset with KC cells and select for patient 2

In [6]:
KC_scc = sc.read_h5ad("../input/preprocessed_data/KC_scc_annData.h5ad") 
KC_scc_p2 = KC_scc[KC_scc.obs['patient']=='P2'] #Reduce data to only patient2 data to manage computational constraints

Add logNormal noise to dataset

In [7]:
scc_p2_noise = add_noise(KC_scc_p2, noise_distribution='logNormal',  scaling_factor=1)

/Users/mo2016/Documents/personal_projects/SCC_helical/scc_helical/functions.py:12: RuntimeWarning: invalid value encountered in sqrt
  gene_stds = np.sqrt((anndata.X.power(2).mean(axis=0) - np.power(gene_means, 2))).flatten()  # Sparse std computation
/Users/mo2016/Library/Caches/pypoetry/virtualenvs/scc-helical--RZZoshj-py3.11/lib/python3.11/site-packages/scipy/sparse/_data.py:73: RuntimeWarning: invalid value encountered in cast
  self.data.astype(dtype, casting=casting, copy=True),


Separate datasets into Tumor, Normal and TSKs (TSK inside tumor)

In [11]:
scc_noise_normal = scc_p2_noise[scc_p2_noise.obs['tum.norm']=='Normal'] #dataset with all cells from normal sample
scc_noise_tumor = scc_p2_noise[scc_p2_noise.obs['tum.norm']=='Tumor'] #dataset with all cells from tumor sample
scc_noise_tumor_TSK = scc_noise_tumor[scc_noise_tumor.obs['level3_celltype']=='TSK'] #dataset with all TSK cells. This dataset will be used for perturbation

Simulate gene deletion of cancer relevant genes to TSK cells understand perturbation effects on TSK oncogenicity.


In [12]:
scc_gene_perturbations = {'ITGB1':'deletion', 'FERMT1':'deletion', 'CD151':'deletion', 'ARPC2':'deletion', 'HSP90B1':'deletion'} #define knockdown
scc_noise_tumor_TSK_knockdown = add_perturbations(scc_noise_tumor_TSK,scc_p2_noise, scc_gene_perturbations) #apply deletions
scc_noise_tumor_TSK_knockdown.obs['tum.norm'] = 'Tumor_knockdown' #modify tumor label to account for knockdown

/Users/mo2016/Library/Caches/pypoetry/virtualenvs/scc-helical--RZZoshj-py3.11/lib/python3.11/site-packages/scipy/sparse/_index.py:151: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Subsample tumor and normal datasets (n=100) to manage computational constraints and merge 4 datasets (tumor, normal and TSK and TSK perturbed)  

In [15]:
shuffled_indices = np.random.permutation(scc_noise_normal.n_obs)
scc_noise_normal = scc_noise_normal[shuffled_indices[:100]]

shuffled_indices = np.random.permutation(scc_noise_tumor.n_obs)
scc_noise_tumor = scc_noise_tumor[shuffled_indices[:100]]

scc_noise_perturbed_merged= concat([scc_noise_tumor_TSK_knockdown, scc_noise_tumor_TSK, scc_noise_normal, scc_noise_tumor ], axis=0, join='outer') #merge three datasets (tumor, normal and TSK)
scc_noise_perturbed_merged.var['gene_name'] = scc_noise_perturbed_merged.var_names
scc_noise_perturbed_merged

Save "perturbed" dataset

In [10]:
scc_noise_perturbed_merged.write_h5ad(f"../output/perturbed_noise_datasets/KC_tumor_TSKknockdown_annData_{len(scc_noise_perturbed_merged)}_noise1.h5ad")


... storing 'tum.norm' as categorical


Delete heavy variables to avoid overloading of RAM

In [11]:
del KC_scc
del scc_gene_perturbations
del scc_noise_normal
del scc_noise_tumor
del scc_noise_tumor_TSK
del scc_noise_tumor_TSK_knockdown
del KC_scc_p2
del scc_p2_noise
del shuffled_indices


# 3. Embeddings with Geneformer_V2_Cancer

Define model to use: geneformer fine tuned with cancer datasets and produce embeddings for "perturbed" dataset

In [16]:
device='mps' #use mac mps for faster inference
geneformer_config_v2_cancer = GeneformerConfig(model_name="gf-12L-95M-i4096-CLcancer", batch_size=1, device=device) #Cancer fine-tuned geneformer
geneformer_v2_cancer= Geneformer(configurer=geneformer_config_v2_cancer) 

cancer_dataset = geneformer_v2_cancer.process_data(scc_noise_perturbed_merged, gene_names="gene_name") #process dataset for geneformer use (normalising, ranking...)
cancer_embeddings = geneformer_v2_cancer.get_embeddings(cancer_dataset) #get embeddings (go from 32738 dimensions (genes) to 512 (embeddings)
print("Cancer-tuned model embeddings shape:", cancer_embeddings.shape)

INFO:helical.utils.downloader:File: '/Users/mo2016/.cache/helical/models/geneformer/v2/gene_median_dictionary.pkl' exists already. File is not overwritten and nothing is downloaded.
INFO:helical.utils.downloader:File saved to: '/Users/mo2016/.cache/helical/models/geneformer/v2/gene_median_dictionary.pkl'
INFO:helical.utils.downloader:File: '/Users/mo2016/.cache/helical/models/geneformer/v2/token_dictionary.pkl' exists already. File is not overwritten and nothing is downloaded.
INFO:helical.utils.downloader:File saved to: '/Users/mo2016/.cache/helical/models/geneformer/v2/token_dictionary.pkl'
INFO:helical.utils.downloader:File: '/Users/mo2016/.cache/helical/models/geneformer/v2/ensembl_mapping_dict.pkl' exists already. File is not overwritten and nothing is downloaded.
INFO:helical.utils.downloader:File saved to: '/Users/mo2016/.cache/helical/models/geneformer/v2/ensembl_mapping_dict.pkl'
INFO:helical.utils.downloader:File: '/Users/mo2016/.cache/helical/models/geneformer/v2/gf-12L-95M-

NameError: name 'scc_noise_perturbed_merged' is not defined

In [14]:
np.save(f"../output/embeddings/KC_tumor_TSKknockdown_embeddings_{len(scc_noise_perturbed_merged)}_noise1.npy", cancer_embeddings)